In [ ]:
import open3d as o3d
import numpy as np

from Eval.Mesh import CreatePointcloudFromDir
from Eval.Clouds.Sphere import CreateBaseSpherePointCloud
from Utils.Viz import VizualiseBaseTargetPointclouds
from Eval.Error import CalculateRMSE
from Utils.Format import MilimeterToMeter, MeterToMilimeter

In [ ]:
# Create Base Clouds for Comparison, Created Programatically for High Accuracy.

# Diameter of Base IRL Sphere: 200mm
baseSphere = CreateBaseSpherePointCloud(diameter = MilimeterToMeter(200), resolution=5000)
baseSphere.paint_uniform_color([0.5, 0.5, 0.5])

In [ ]:
# Current Technique and Data in Review

eval = 'Kinect' # Either Kinect, PolyCam or Point_E

evalDataDir = f'../data/{eval}/sphere/sphere.ply'

In [ ]:
# Get the Obtained Data Clouds for Comparison, on the Base Programatically Created Clouds.

collectedSphere = CreatePointcloudFromDir(evalDataDir)
collectedSphere.paint_uniform_color([1, 0, 0])

#collectedSphere.scale(MilimeterToMeter(200), collectedSphere.get_center())

In [ ]:
# Perform ICP Registration to Obtain a Transformation Matrix
# This allows for more accurate comparison of the clouds, by making them similar positioning.


icpRegResults = o3d.pipelines.registration.registration_icp(baseSphere, 
                                                            collectedSphere, 
                                                            0.1, 
                                                            np.identity(4), 
                                                            o3d.pipelines.registration.TransformationEstimationPointToPoint(), 
                                                            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000))

print(icpRegResults)

# Apply the Transformation Matrix to the Obtained Data Clouds
baseSphere.transform(icpRegResults.transformation)

In [ ]:
# Quick Visualization of both the Obtained Cloud and the Base Cloud.

VizualiseBaseTargetPointclouds(baseSphere, collectedSphere)

In [ ]:
# Calculate the RMSE between the two clouds.

rmse = CalculateRMSE(baseSphere, collectedSphere)

# Convert to mm
rmse *= 1000

# Added * 1000 to convert to mm
print(f'RMSE (mm): {rmse}')

### Save and Plot RMSE Data

In [ ]:
from Eval.Plotting import plot_technique_rmse
from Utils.Data import JSON

In [ ]:
jsonFileDir = "../data/techniques-rmse-diagram.json"

rmse_data = JSON(jsonFileDir)

In [ ]:
rmse_data.data[eval]["sphere"].update({
    "rmse": rmse,
    "inlier_rmse": MeterToMilimeter(icpRegResults.inlier_rmse) # Convert to mm
})

rmse_data.print()

rmse_data.save()

In [ ]:
# Create Figure Based on Current Dataset

fig = plot_technique_rmse(rmse_data.data, upper_lim=12) # 1.2cm

fig.show()

In [ ]:
# Save Figure

fig.savefig("../data/techniques-rmse-diagram.png", format="png", dpi=300)